In [1]:
from ksql import KSQLAPI
client = KSQLAPI('http://localhost:8088/')

In [2]:
client.ksql('show tables')

[{'@type': 'tables',
  'statementText': 'show tables;',
  'tables': [],
  'warnings': []}]

In [28]:
raw_message = {"messageId": 7, "sessionId": "04408b87-17fb-4fd3-a58c-025acd03f8b8", "deviceId": "86a5b0e3-6e06-40e2-b226-5a72bd39b65b", "payload": [{"name": "gyroscopeuncalibrated", "time": "1679321880545268000", "values": {"z": 0.00726457079872489, "y": -0.00706856232136488, "x": -0.02882285974919796}}, {"values": {"z": -49.446685791015625, "y": -3.390331268310547, "x": -34.99919891357422}, "name": "magnetometer", "time": "1679321880545268000", "accuracy": "2"}, {"name": "gyroscope", "time": "1679321880545268000", "values": {"z": 0.006179125513881445, "y": -0.00965726189315319, "x": -0.02642429992556572}}, {"name": "accelerometer", "time": "1679321880545268000", "values": {"z": -0.06621931583881378, "y": 0.03285772898495197, "x": -0.06695471744127571}}, {"name": "magnetometeruncalibrated", "time": "1679321880561757200", "values": {"z": -376.43121337890625, "y": -27.739212036132812, "x": -113.55758666992188}}, {"name": "accelerometeruncalibrated", "time": "1679321880560308000", "values": {"z": -0.8322296142578125, "y": -0.5608978271484375, "x": -0.06060791015625}}, {"name": "gyroscopeuncalibrated", "time": "1679321880565322200", "values": {"z": -0.0027001791168004274, "y": -0.010466222651302814, "x": -0.030856182798743248}}, {"values": {"z": -50.064788818359375, "y": -3.322660446166992, "x": -35.011444091796875}, "name": "magnetometer", "time": "1679321880565322200", "accuracy": "2"}, {"name": "gyroscope", "time": "1679321880565322200", "values": {"z": -0.003785535227507353, "y": -0.01305432803928852, "x": -0.02845705859363079}}, {"name": "accelerometer", "time": "1679321880565322200", "values": {"z": -0.07442424960434436, "y": 0.041241526448726654, "x": 0.0011834010982885955}}, {"name": "magnetometeruncalibrated", "time": "1679321880581787100", "values": {"z": -375.5218505859375, "y": -28.142807006835938, "x": -113.31315612792969}}, {"name": "accelerometeruncalibrated", "time": "1679321880580362000", "values": {"z": -0.821380615234375, "y": -0.563140869140625, "x": -0.0584564208984375}}, {"name": "gyroscopeuncalibrated", "time": "1679321880585375000", "values": {"z": -0.012527243234217167, "y": -0.009367935359477997, "x": 0.00023462448734790087}}, {"values": {"z": -49.419586181640625, "y": -3.589061737060547, "x": -34.81123352050781}, "name": "magnetometer", "time": "1679321880585375000", "accuracy": "2"}, {"name": "gyroscope", "time": "1679321880585375000", "values": {"z": -0.013612422160804272, "y": -0.011954856105148792, "x": 0.0026348773390054703}}, {"name": "accelerometer", "time": "1679321880585375000", "values": {"z": 0.02763619494438171, "y": 0.038030163186788556, "x": -0.007290120475552976}}, {"name": "magnetometeruncalibrated", "time": "1679321880601815000", "values": {"z": -375.5806884765625, "y": -28.14788818359375, "x": -113.55940246582031}}, {"name": "accelerometeruncalibrated", "time": "1679321880600415000", "values": {"z": -0.8106689453125, "y": -0.5614471435546875, "x": -0.059600830078125}}, {"name": "gyroscopeuncalibrated", "time": "1679321880605429000", "values": {"z": -0.0057534934021532536, "y": -0.008242483250796795, "x": 0.0199835617095232}}, {"values": {"z": -49.492645263671875, "y": -3.608278274536133, "x": -34.988059997558594}, "name": "magnetometer", "time": "1679321880605429000", "accuracy": "2"}, {"name": "gyroscope", "time": "1679321880605429000", "values": {"z": -0.006838672794401646, "y": -0.01082940399646759, "x": 0.022383814677596092}}, {"name": "accelerometer", "time": "1679321880605429000", "values": {"z": 0.11194061805009842, "y": 0.04098725942671299, "x": -0.01740154544953257}}, {"name": "magnetometeruncalibrated", "time": "1679321880621843200", "values": {"z": -376.1710205078125, "y": -27.74957275390625, "x": -113.45982360839844}}, {"name": "accelerometeruncalibrated", "time": "1679321880620469000", "values": {"z": -0.816558837890625, "y": -0.5629119873046875, "x": -0.0582122802734375}}, {"name": "gyroscopeuncalibrated", "time": "1679321880625482000", "values": {"z": 0.004927380476146936, "y": -0.008587096817791462, "x": 0.002569950418546796}}, {"values": {"z": -49.933197021484375, "y": -3.2871265411376953, "x": -34.912803649902344}, "name": "magnetometer", "time": "1679321880625482000", "accuracy": "2"}, {"name": "gyroscope", "time": "1679321880625482000", "values": {"z": 0.0038422669749706984, "y": -0.011173573322594166, "x": 0.0049706255085766315}}, {"name": "accelerometer", "time": "1679321880625482000", "values": {"z": 0.03914835356175899, "y": 0.02141571299433708, "x": 0.012811295049823819}}, {"name": "magnetometeruncalibrated", "time": "1679321880641870000", "values": {"z": -375.2215576171875, "y": -28.21014404296875, "x": -113.31687927246094}}, {"name": "accelerometeruncalibrated", "time": "1679321880640522000", "values": {"z": -0.826019287109375, "y": -0.562347412109375, "x": -0.0573577880859375}}, {"name": "gyroscopeuncalibrated", "time": "1679321880645536000", "values": {"z": 0.00714792450889945, "y": -0.010241186246275902, "x": -0.039021968841552734}}, {"values": {"z": -49.272491455078125, "y": -3.6395225524902344, "x": -34.79270935058594}, "name": "magnetometer", "time": "1679321880645536000",

 "accuracy": "2"}, {"name": "gyroscope", "time": "1679321880645536000", "values": {"z": 0.0060628545470535755, "y": -0.012827366590499878, "x": -0.036621011793613434}}, {"name": "accelerometer", "time": "1679321880645536000", "values": {"z": -0.0322159239500761, "y": 0.046015316721796984, "x": 0.01560731897652149}}, {"name": "magnetometeruncalibrated", "time": "1679321880661900000", "values": {"z": -376.184814453125, "y": -27.70404052734375, "x": -113.08076477050781}}, {"name": "accelerometeruncalibrated", "time": "1679321880660576000", "values": {"z": -0.8188629150390625, "y": -0.5643768310546875, "x": -0.057342529296875}}, {"name": "gyroscopeuncalibrated", "time": "1679321880665589000", "values": {"z": -0.00509968725964427, "y": 0.002429601736366749, "x": -0.03797667846083641}}, {"values": {"z": -49.8116455078125, "y": -3.265390396118164, "x": -34.59791564941406}, "name": "magnetometer", "time": "1679321880665589000", "accuracy": "2"}, {"name": "gyroscope", "time": "1679321880665589000", "values": {"z": -0.0061851548962295055, "y": -0.00015924591571092606, "x": -0.03557825833559036}}, {"name": "accelerometer", "time": "1679321880665589000", "values": {"z": 0.05212064785957336, "y": -0.007556698989868164, "x": 0.0010989011526107788}}, {"name": "magnetometeruncalibrated", "time": "1679321880681930000", "values": {"z": -374.9447021484375, "y": -28.029434204101562, "x": -113.47869873046875}}, {"name": "accelerometeruncalibrated", "time": "1679321880680629000", "values": {"z": -0.8147735595703125, "y": -0.5639190673828125, "x": -0.0655059814453125}}, {"name": "gyroscopeuncalibrated", "time": "1679321880685643000", "values": {"z": -0.002104163635522127, "y": 0.009623332880437374, "x": -0.010124272666871548}}, {"values": {"z": -48.920654296875, "y": -3.5312671661376953, "x": -34.900390625}, "name": "magnetometer", "time": "1679321880685643000", "accuracy": "2"}, {"name": "gyroscope", "time": "1679321880685643000", "values": {"z": -0.0031894310377538204, "y": 0.007035819813609123, "x": -0.0077245840802788734}}, {"name": "accelerometer", "time": "1679321880685643000", "values": {"z": 0.07526537660360336, "y": 0.01802197890281677, "x": -0.07095956916846334}}, {"name": "magnetometeruncalibrated", "time": "1679321880701958000", "values": {"z": -375.4111633300781, "y": -28.206314086914062, "x": -113.13569641113281}}, {"name": "accelerometeruncalibrated", "time": "1679321880700683000", "values": {"z": -0.8227691650390625, "y": -0.564605712890625, "x": -0.06219482421875}}, {"name": "gyroscopeuncalibrated", "time": "1679321880705696000", "values": {"z": 0.006340453866869211, "y": -0.004033357370644808, "x": 0.007348460145294666}}, {"values": {"z": -49.253265380859375, "y": -3.625873565673828, "x": -34.67518615722656}, "name": "magnetometer", "time": "1679321880705696000", "accuracy": "2"}, {"name": "gyroscope", "time": "1679321880705696000", "values": {"z": 0.005255318246781826, "y": -0.0066199819557368755, "x": 0.009748994372785091}}, {"name": "accelerometer", "time": "1679321880705696000", "values": {"z": -0.024487375524640084, "y": 0.0020136779099702833, "x": -0.028535299208573995}}, {"name": "magnetometeruncalibrated", "time": "1679321880721987000", "values": {"z": -375.66741943359375, "y": -27.824905395507812, "x": -113.560546875}}, {"name": "accelerometeruncalibrated", "time": "1679321880720736000", "values": {"z": -0.82843017578125, "y": -0.5604095458984375, "x": -0.0599822998046875}}, {"name": "gyroscopeuncalibrated", "time": "1679321880725750000", "values": {"z": 0.006897853687405586, "y": -0.016138223931193352, "x": 0.005641906522214413}}, {"values": {"z": -49.515838623046875, "y": -3.426464080810547, "x": -34.96112060546875}, "name": "magnetometer", "time": "1679321880725750000", "accuracy": "2"}, {"name": "gyroscope", "time": "1679321880725750000", "values": {"z": 0.005812408402562141, "y": -0.018726922571659088, "x": 0.008040466345846653}}, {"name": "accelerometer", "time": "1679321880725750000", "values": {"z": -0.04477320770621299, "y": 0.06440846702456474, "x": 0.002697751183994114}}, {"name": "magnetometeruncalibrated", "time": "1679321880742017000", "values": {"z": -375.767822265625, "y": -28.010635375976562, "x": -113.5001220703125}}, {"name": "accelerometeruncalibrated", "time": "1679321880740790000", "values": {"z": -0.8206329345703125, "y": -0.553924560546875, "x": -0.058319091796875}}, {"name": "gyroscopeuncalibrated", "time": "1679321880745803000", "values": {"z": -0.0034975295420736074, "y": -0.013859889470040798, "x": -0.010239588096737862}}]}

In [29]:
def flatten_dict(nested_dict, parent_key='', sep='_'):
    """
    This function takes in a nested dictionary and returns a flattened dictionary.
    """
    items = []
    for key, value in nested_dict.items():
        new_key = f"{parent_key}{sep}{key}" if parent_key else key
        if isinstance(value, dict):
            items.extend(flatten_dict(value, new_key, sep=sep).items())
        else:
            items.append((new_key, value))
    return dict(items)

In [43]:
message_info = raw_message.copy()
message_info.pop('payload')

transformed_payload = [{**flatten_dict(old_dict), **message_info} for old_dict in raw_message['payload']]
transformed_payload

[{'name': 'gyroscopeuncalibrated',
  'time': '1679321880545268000',
  'values_z': 0.00726457079872489,
  'values_y': -0.00706856232136488,
  'values_x': -0.02882285974919796,
  'messageId': 7,
  'sessionId': '04408b87-17fb-4fd3-a58c-025acd03f8b8',
  'deviceId': '86a5b0e3-6e06-40e2-b226-5a72bd39b65b'},
 {'values_z': -49.446685791015625,
  'values_y': -3.390331268310547,
  'values_x': -34.99919891357422,
  'name': 'magnetometer',
  'time': '1679321880545268000',
  'accuracy': '2',
  'messageId': 7,
  'sessionId': '04408b87-17fb-4fd3-a58c-025acd03f8b8',
  'deviceId': '86a5b0e3-6e06-40e2-b226-5a72bd39b65b'},
 {'name': 'gyroscope',
  'time': '1679321880545268000',
  'values_z': 0.006179125513881445,
  'values_y': -0.00965726189315319,
  'values_x': -0.02642429992556572,
  'messageId': 7,
  'sessionId': '04408b87-17fb-4fd3-a58c-025acd03f8b8',
  'deviceId': '86a5b0e3-6e06-40e2-b226-5a72bd39b65b'},
 {'name': 'accelerometer',
  'time': '1679321880545268000',
  'values_z': -0.06621931583881378,
